In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml
!pip install cloudpickle
!pip install -U cloudpickle

  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached scikit_learn-1.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 3.0.0
    Uninstalling cloudpickle-3.0.0:
      Successfully uninstalled cloudpickle-3.0.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.1
    Uninstalling scikit-learn-1.2.1:
      Successfully uninstalled scikit-learn-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mosaic-utils 1.0.2 requires scikit-learn==1.2.1; python_version >= "3.8", but you have scikit-learn 1.3.2 which is incompatible.
refractml 1.0.8 r

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [2]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [3]:
df = 'ATTRITION_MASTER_TABLE'

In [4]:
sf_df = my_session.sql("select * from {}".format(df))

In [5]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [6]:
pandas_df

,SALARY,EMPLOYEE_ID,JOB_SATISFACTION,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,OVER_TIME,RELATIONSHIP_SATISFACTION,CHURN_STATUS_TABLE,CHURN_VALUE_TABLE,CITY,...,RETENTION,SUM_OF_TENURE,SUM_OF_AGE,JOB_STARTDATE,JOB_ENDDATE,OVERTIME_HOURS,WORK_LIFE_BALANCE,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION,USER_ID
0,43118.65,94215814,High,20,Outstanding,No,Medium,FALSE,0,Charlotte,...,0,0,0,2016-06-01,2019-08-01,4,Best,Travel Rarely,Medium,334493477
1,32365.59,20227353,High,14,Outstanding,No,Medium,TRUE,1,Charlotte,...,50,9366,2178,2016-06-01,2021-06-01,1,Better,Travel Rarely,High,89260492
2,40808.18,72815065,Very High,12,Low,No,Very High,TRUE,1,Charlotte,...,0,0,0,2015-05-01,None,7,Best,Travel Frequently,Low,713161033
3,22281.58,15048356,Low,17,Good,No,Very High,TRUE,1,Charlotte,...,0,0,0,2016-10-01,2017-07-01,16,Bad,Travel Rarely,Medium,844498976
4,38163.65,91151756,Medium,21,Low,Yes,Medium,FALSE,0,Charlotte,...,0,0,0,2016-06-01,2019-08-01,4,Better,Travel Rarely,Very High,334493477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,99171.50,11954788,Very High,11,Excellent,No,Low,FALSE,0,Houston,...,97,0,0,2021-12-01,2023-05-01,14,Bad,Travel Frequently,Low,492160448
299996,93665.50,65800600,High,12,Low,Yes,High,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-05-01,14,Better,Travel Rarely,High,492160448
299997,98761.50,78774518,High,14,Excellent,No,Low,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-05-01,14,Better,Non-Travel,Medium,492160448
299998,98597.50,53352261,Medium,25,Good,No,High,FALSE,0,Houston,...,0,0,0,2021-12-01,2023-05-01,14,Bad,Travel Rarely,Very High,492160448


In [7]:
print(pandas_df.isnull().sum())

SALARY                                             0
EMPLOYEE_ID                                        0
JOB_SATISFACTION                                   0
PERCENTAGE_SALARY_HIKE                             0
PERFORMANCE_RATING                                 0
OVER_TIME                                          0
RELATIONSHIP_SATISFACTION                          0
CHURN_STATUS_TABLE                                 0
CHURN_VALUE_TABLE                                  0
CITY                                               0
DISTANCE                                           0
DEGREE_CLEAN                                       0
ETHNICITY                                          0
MARITAL_STATUS                                     0
ROLE                                               0
COMPANY_NAME                                       0
ORGANIZATION_TYPE                                  0
ORGANIZATION_OWNERSHIP                             0
STATE                                         

In [8]:
Original_df = pandas_df.dropna()

In [9]:
print(Original_df.isnull().sum())

SALARY                                         0
EMPLOYEE_ID                                    0
JOB_SATISFACTION                               0
PERCENTAGE_SALARY_HIKE                         0
PERFORMANCE_RATING                             0
OVER_TIME                                      0
RELATIONSHIP_SATISFACTION                      0
CHURN_STATUS_TABLE                             0
CHURN_VALUE_TABLE                              0
CITY                                           0
DISTANCE                                       0
DEGREE_CLEAN                                   0
ETHNICITY                                      0
MARITAL_STATUS                                 0
ROLE                                           0
COMPANY_NAME                                   0
ORGANIZATION_TYPE                              0
ORGANIZATION_OWNERSHIP                         0
STATE                                          0
COUNTRY                                        0
GENDER              

In [10]:
df = Original_df.drop(["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE","CHURN_OTHER","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","TOTAL_EMPLOYEE","RETENTION","SUM_OF_TENURE","SUM_OF_AGE"], axis = 1)

In [11]:
df

,SALARY,JOB_SATISFACTION,PERCENTAGE_SALARY_HIKE,PERFORMANCE_RATING,OVER_TIME,RELATIONSHIP_SATISFACTION,CHURN_STATUS_TABLE,CHURN_VALUE_TABLE,CITY,DISTANCE,...,GENDER,TENURE_MONTHS,BIRTH_YEAR,AGE,SENIORITY,MONTHS_AFTER_COLLEGE,OVERTIME_HOURS,WORK_LIFE_BALANCE,BUSINESS_TRAVEL,ENVIRONMENT_SATISFACTION
0,43118.65,High,20,Outstanding,No,Medium,FALSE,0,Charlotte,<2 miles,...,Male,35,1981,43,1,99,4,Best,Travel Rarely,Medium
1,32365.59,High,14,Outstanding,No,Medium,TRUE,1,Charlotte,<2 miles,...,Female,59,1991,33,1,63,1,Better,Travel Rarely,High
3,22281.58,Low,17,Good,No,Very High,TRUE,1,Charlotte,<2 miles,...,Male,6,1992,32,1,7,16,Bad,Travel Rarely,Medium
4,38163.65,Medium,21,Low,Yes,Medium,FALSE,0,Charlotte,<2 miles,...,Male,35,1981,43,1,99,4,Better,Travel Rarely,Very High
6,27599.60,High,14,Outstanding,Yes,Low,TRUE,1,Dallas,>10 miles,...,Female,2,1987,37,1,23,20,Best,Travel Rarely,Very High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,99171.50,Very High,11,Excellent,No,Low,FALSE,0,Houston,>10 miles,...,Male,15,1990,34,2,9,14,Bad,Travel Frequently,Low
299996,93665.50,High,12,Low,Yes,High,FALSE,0,Houston,>10 miles,...,Female,15,1990,34,2,9,14,Better,Travel Rarely,High
299997,98761.50,High,14,Excellent,No,Low,FALSE,0,Houston,>10 miles,...,Female,15,1990,34,2,9,14,Better,Non-Travel,Medium
299998,98597.50,Medium,25,Good,No,High,FALSE,0,Houston,>10 miles,...,Female,15,1990,34,2,9,14,Bad,Travel Rarely,Very High


In [12]:
CATEGORICAL_COLUMNS = ["ROLE","ETHNICITY","ORGANIZATION_TYPE", "ORGANIZATION_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE","COUNTRY","GENDER",
                       "BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARITAL_STATUS","OVER_TIME","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE","CHURN_STATUS_TABLE","DEGREE_CLEAN"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE","OVERTIME_HOURS","PERCENTAGE_SALARY_HIKE"]
LABEL_COLUMNS = ["CHURN_VALUE_TABLE"]
DROPPED_COLUMNS = ["USER_ID", "EMPLOYEE_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE","CHURN_OTHER","PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH","PEOPLE_JOINED_AND_NEVER_LEFT","TOTAL_EMPLOYEE","RETENTION","SUM_OF_TENURE","SUM_OF_AGE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [13]:
# Filter feature columns
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS
feature_columns = [col for col in feature_columns if col in Original_df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in Original_df.columns]
 
# Split data into features and labels
X = Original_df[feature_columns + DROPPED_COLUMNS]
y = Original_df[LABEL_COLUMNS].values.ravel()  # Flatten to 1D array for consistency

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [16]:
X_test

,ROLE,ETHNICITY,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,COMPANY_NAME,CITY,STATE,DISTANCE,COUNTRY,GENDER,...,JOB_STARTDATE,JOB_ENDDATE,SCHOOL_ENDDATE,CHURN_OTHER,PEOPLE_JOINED_BEFORE_AND_LEFT_IN_THIS_MONTH,PEOPLE_JOINED_AND_NEVER_LEFT,TOTAL_EMPLOYEE,RETENTION,SUM_OF_TENURE,SUM_OF_AGE
264980,Warehouse Associate,Hispanic,Environmental,Non-Profit,GreenWorld Foundation,Chicago,Illinois,>10 miles,United States,Male,...,2021-09-01,2023-11-01,2019-01-01,0,0,0,0,0,0,0
141529,Warehouse Associate,White,IT Services,Public,GreenByte Solutions,Jacksonville,Florida,2-5 miles,United States,Male,...,2022-04-01,2024-04-01,2020-01-01,350,0,206,206,387,29034,7416
235142,Recruiter,API,Transportation,Private,EcoTransport LLC,Orlando,Florida,5-10 miles,United States,Female,...,2021-04-01,2022-09-01,2018-01-01,0,0,0,0,0,0,0
96658,Employee Relations Manager,White,Technology,Private,Innovatech Ltd.,Rochester,New York,5-10 miles,United States,Male,...,2018-12-01,2020-10-01,2016-01-01,0,0,0,0,0,0,0
89796,Creative Director,Multiple,IT Services,Public,GreenByte Solutions,Boston,Massachusetts,<2 miles,United States,Female,...,2022-09-01,2023-06-01,2020-01-01,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227742,Research Scientist,White,Environmental,Non-Profit,GreenWorld Foundation,Tampa,Florida,>10 miles,United States,Male,...,2015-09-01,2016-05-01,2011-01-01,0,0,0,0,0,0,0
13711,Compliance Manager,Hispanic,Healthcare,Non-Profit,MetroHealth Systems,Tampa,Florida,2-5 miles,United States,Female,...,2018-11-01,2021-10-01,2013-01-01,0,0,0,0,0,0,0
95086,Accounting Manager,API,IT Services,Public,GreenByte Solutions,Jacksonville,Florida,<2 miles,United States,Female,...,2018-03-01,2019-03-01,2010-01-01,350,0,206,206,387,29034,7416
252668,Visual Designer,Multiple,Financial Services,Cooperative,RuralCooperative Bank,Worcester,Massachusetts,>10 miles,United States,Male,...,2020-10-01,2024-06-01,2018-01-01,0,0,0,0,0,0,0


In [17]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
 
# Define transformers
categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)
 
numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)
 
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)
 
# Create pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)
result = pipeline.predict(X_test)

In [18]:
result

array([0, 0, 1, ..., 1, 0, 0])

In [19]:
from joblib import dump, load
filename = "Attrition.joblib"
dump(pipeline, filename)

['Attrition.joblib']

In [20]:
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)

In [21]:
from refractml import *

from refractml.constants import MLModelFlavours

In [22]:
@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data_json = eval(payload)
    data = pd.DataFrame([data_json])
    prediction = str(model.predict(data)[0])
    return prediction

In [23]:
import requests
import datetime
import pandas as pd 
payload = str(X_test.iloc[123].to_dict())
req = requests.Request()
req.json = {"payload": payload}

print(score(pipeline, req))

1


In [24]:
req.json

{'payload': "{'ROLE': 'Warehouse Associate', 'ETHNICITY': 'White', 'ORGANIZATION_TYPE': 'E-commerce', 'ORGANIZATION_OWNERSHIP': 'Private', 'COMPANY_NAME': 'BrightStart Ventures', 'CITY': 'Cleveland', 'STATE': 'Ohio', 'DISTANCE': '5-10 miles', 'COUNTRY': 'United States', 'GENDER': 'Female', 'BUSINESS_TRAVEL': 'Travel Rarely', 'ENVIRONMENT_SATISFACTION': 'Low', 'JOB_SATISFACTION': 'High', 'MARITAL_STATUS': 'Married', 'OVER_TIME': 'Yes', 'PERFORMANCE_RATING': 'Outstanding', 'RELATIONSHIP_SATISFACTION': 'High', 'WORK_LIFE_BALANCE': 'Better', 'CHURN_STATUS_TABLE': 'TRUE', 'DEGREE_CLEAN': 'Other', 'SALARY': 36707.29, 'SENIORITY': 1, 'TENURE_MONTHS': 20, 'MONTHS_AFTER_COLLEGE': 55, 'BIRTH_YEAR': 1988, 'AGE': 36, 'OVERTIME_HOURS': 10, 'PERCENTAGE_SALARY_HIKE': 13, 'USER_ID': 698929203, 'EMPLOYEE_ID': '58831977', 'JOB_STARTDATE': datetime.date(2017, 10, 1), 'JOB_ENDDATE': datetime.date(2019, 8, 1), 'SCHOOL_ENDDATE': datetime.date(2011, 1, 31), 'CHURN_OTHER': 0, 'PEOPLE_JOINED_BEFORE_AND_LEFT_IN

In [3]:
## registering the model.
from fosforml import register_model

register_model(
  model_obj=model,
  session=my_session,
  x_train=x_train,
  y_train=y_train,
  x_test=x_test,
  y_test=y_test,
  y_pred=y_pred,
  source="Notebook",
  dataset_name="ATTRITION_MASTER_TABLE",
  dataset_source="Snowflake",
  name="attrition_master_table",
  description="This is a Model for the analyzing attrition",
  flavour="sklearn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)


NameError: name 'model' is not defined

In [2]:
from fosforml import register_model

  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 1.6.0
    Uninstalling cloudpickle-1.6.0:
      Successfully uninstalled cloudpickle-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fosforml 1.1.7 requires cloudpickle==2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
fosforml 1.1.7 requires scikit-learn==1.3.2, but you have scikit-learn 1.2.1 which is incompatible.
refractml 1.0.8 requires cloudpickle==1.6.0, but you have cloudpickle 3.0.0 which is incompatible.
snowflake-snowpark-python 1.21.1 requires cloudpickle!=2.1.0,!=2.2.0,<=2.2.1,>=1.6.0; python_version < "3.11", but you have cloudpickle 3.0.0 which is incompatible.


In [ ]:
payload  = {"payload": X_test.iloc[0].to_dict()}
payload

In [ ]:
result = pipeline.predict(X_test)
result_prob = pipeline.predict_proba(X_test)
pred_df = X_test.copy()
result = result
result_prob = result_prob
pred_df["PREDICTION"] = result
pred_df["PROB"] = result_prob[:, 1]

In [ ]:
from sklearn.metrics import classification_report, log_loss, roc_auc_score
 
# Check lengths
print("Length of y_test:", len(y_test))
print("Length of y_pred:", len(y_pred))
 
# Print classification report
print(classification_report(y_test, y_pred))
 
# Calculate accuracy
accuracy = sum(y_test == y_pred) / len(y_test)
print("Accuracy:", accuracy)
 
# Calculate additional metrics
log_loss_value = log_loss(y_test, y_prob)
roc_auc = roc_auc_score(y_test, y_prob[:, 1])  # Assuming class 1 is the positive class
 
print("Log Loss:", log_loss_value)
print("ROC AUC Score:", roc_auc)

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import seaborn as sns
ns_probs = [0 for _ in range(len(y_test))]
ns_auc = roc_auc_score(y_test, ns_probs)
lr_auc = roc_auc_score(y_test, pred_df["PROB"])
ns_fpr, ns_tpr, _ = roc_curve(y_test, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(y_test, pred_df["PROB"])
# plot the roc curve for the model
plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
plt.plot(lr_fpr, lr_tpr, marker='.', label='Radnomforest')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
lr_precision, lr_recall, _ = precision_recall_curve(y_test, pred_df["PROB"])
plt.plot(lr_recall, lr_precision, marker='.', label='Randomforest')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
cm = confusion_matrix(y_test,y_pred)
 
#Plot the confusion matrix.
sns.heatmap(cm,
            annot=True,
            fmt='g',
            xticklabels=['Not Churn','Churn'],
            yticklabels=['Not CHurn','Churn'])
plt.ylabel('Prediction',fontsize=13)
plt.xlabel('Actual',fontsize=13)
plt.title('Confusion Matrix',fontsize=17)
plt.show()

In [ ]:
scores_data = pd.DataFrame({"ns_probs":ns_probs,
                            "y_test":y_test,
                            "y_pred":pred_df["PREDICTION"],
                            "act_probs":pred_df["PROB"]})

In [ ]:
scores_data

In [ ]:
scores_data.to_csv("/data/scores.csv", index=False)

In [ ]:
# Define or use the prediction function
def model_prediction_score_func(dataframe):
    # Ensure 'dataframe' has the correct features required by the model
    return pipeline.predict(dataframe)  # Use your trained pipeline/model here
 
# Assuming df is your input DataFrame with the necessary features
Original_df['Model_Output'] = model_prediction_score_func(Original_df)
 
# If you have a DataFrame with test data and you want to merge predictions
# Assuming X_test is the DataFrame for which predictions are made
#X_test_with_predictions = X_test.copy()
#X_test_with_predictions['Model_Output'] = model_prediction_score_func(X_test)
 
# Display the first few rows to verify
#print(X_test_with_predictions.head())

In [ ]:
Original_df

In [ ]:
Original_df.to_csv("/data/Hr_Attrition_Data.csv", index=False)

In [ ]:
con = snowflake.get_connection("HR_ATTRITION")

In [ ]:
import pandas as pd
if 'Model_Output' in Original_df:
    Original_df['Model_Output_Values'] = Original_df['Model_Output'].astype(bool)
print(Original_df.head())


In [ ]:
file = 'HR_Attrition_Data.csv'
Original_df.to_csv(file , index=False)

In [ ]:
file

In [ ]:
snowflake.execute_query(f"PUT file://{file} @HR_ANALYTIC.PUBLIC.HR_ATTRITION_STAGE")